In [2]:
import pandas as pd
import numpy as np

In [3]:
import sys
sys.path.append('../src')
from visual_corr import cat_cat, cont_cont, cat_cont, cont_label, cat_label, corr, drawRow
from utils import no_accent_vietnamese, FeatureFromColumn
from split_train_val import spanOneHot2dict, new_column, calcWeight, features2Dataset,\
overSampling, dataset2Folds, auc_plot, auc_value

In [4]:
from features_desc1 import notnull_list
from features_desc0 import null_list

In [5]:
Finit = pd.read_csv('../data/train.csv')
Finit1 = Finit[Finit.isnull().sum(axis=1) <= 40]
Finit0 = Finit[Finit.isnull().sum(axis=1) > 40]

/home/loitg/miniconda2/envs/creditscore/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
Fprovince_ext = pd.read_csv('./Fprovince_ext.csv')
Fprovince_ext_1 = Fprovince_ext.loc[Finit1.index]
Fprovince_ext_0 = Fprovince_ext.loc[Finit0.index]


F7ext = pd.read_csv('./F7ext.csv')
F7ext1 = F7ext.loc[Finit1.index]
F7ext0 = F7ext.loc[Finit0.index]

local1 = pd.concat([Finit1.drop(columns=['label']), Fprovince_ext_1.drop(columns=['label', 'province']), F7ext1], axis=1)
local0 = pd.concat([Finit0.drop(columns=['label']), Fprovince_ext_0.drop(columns=['label', 'province']), F7ext0], axis=1)

In [78]:
selected_list = [f for f in notnull_list if f.isSure]
cleanedds = features2Dataset(local1, 'label', selected_list, allSingleForNN=True)

0_FIELD_1
1_age_source2
2_age_source1
3_FIELD_5
4_FIELD_6
5_FIELD_7
6_FIELD_8
7_FIELD_10
8_FIELD_11
9_FIELD_12
10_FIELD_14
11_FIELD_15
12_FIELD_17
13_FIELD_18
14_FIELD_19
15_FIELD_20
16_FIELD_22
17_FIELD_25
18_FIELD_26
19_FIELD_29
20_FIELD_30
21_FIELD_31
22_FIELD_46
23_FIELD_47
24_FIELD_48
25_FIELD_51
26_FIELD_52
27_FIELD_53
28_FIELD_54
29_FIELD_56
30_FIELD_57
31_vung
32_CH
33_f7count


In [83]:
def doo(cleanedds, kfold=10, seed=8, **kwargs):
    folds = dataset2Folds(cleanedds, 'label', kfold=kfold, seed=seed)
    predicted_labels = []
    model = None
    for i in range(len(folds)):
        print('.', end='')
        from xgboost import XGBClassifier
        model = XGBClassifier(**kwargs)

#         from sklearn.ensemble import GradientBoostingClassifier
#         model = GradientBoostingClassifier(n_estimators=50, learning_ratea=0.05, max_depth=4, random_state=seed)

#         from sklearn.neural_network import MLPClassifier
#         model = MLPClassifier(hidden_layer_sizes=(kwargs['l1'],kwargs['l2']), max_iter=1, activation = 'relu',solver='adam',random_state=seed)

        train_idx, test_idx = folds[i]
        train_ds = cleanedds.iloc[train_idx]
        #train_ds = overSampling(train_ds, 'label',kwargs['ratio'])True
        #train_weight = calcWeight(train_ds['label'])
        train_weight = (train_ds['label']+0.009)
        other_columns = [c for c in cleanedds.columns if c != 'label']
        model.fit(train_ds[other_columns], train_ds['label'], sample_weight=train_weight)

        test_ds = cleanedds.iloc[test_idx]
        predicted_label = model.predict_proba(test_ds[other_columns])
        predicted_labels.append(pd.DataFrame(index=test_ds.index, data=predicted_label, columns=[0,1]))


    predicted = pd.concat(predicted_labels).sort_index()
    return auc_value(cleanedds['label'], predicted[1])

In [80]:
cleanedds.shape

(20322, 65)

In [79]:
import warnings
warnings.filterwarnings('ignore')
doo(cleanedds, kfold=10, seed=0)

.

KeyError: 'l1'

In [84]:
params  = {'l1':np.linspace(30, 40, 3),\
          'l2':np.linspace(4,12,5),\
          'ratio':np.linspace(0.4,0.9,3)}

import time
def ttt(fixed, varied, ret, kfold):
    if len(varied) > 0:
        k = next(iter(varied))
        lspace = varied.pop(k)
        for v in lspace:
            if v.is_integer():
                v = int(v)
            newfixed = fixed.copy()
            newfixed[k] = v
            ttt(newfixed, varied.copy(), ret, kfold)
    else:
        vs = []
        millis = int(round(time.time() * 1000))
        for seed in range(millis, millis + 4):
            v = doo(cleanedds, kfold=kfold, seed=seed, **fixed)
            vs.append(v)
        vs = np.array(vs)
        fixed.update({'mean':vs.mean()*100, 'std':vs.std()*100})
        print()
        print(fixed)
        ret.append(fixed)
          
ret = []
ttt({} , params, ret, kfold=10)       


def scanParams()

........................................
{'l1': 30, 'l2': 4, 'ratio': 0.4, 'mean': 25.44693251835597, 'std': 1.533519669862199}
........................................
{'l1': 30, 'l2': 4, 'ratio': 0.65, 'mean': 26.651646203199842, 'std': 1.4286742709193936}
........................................
{'l1': 30, 'l2': 4, 'ratio': 0.9, 'mean': 27.57627585053187, 'std': 1.353542547608702}
........................................
{'l1': 30, 'l2': 6, 'ratio': 0.4, 'mean': 26.506758637462063, 'std': 1.451624717731776}
........................................
{'l1': 30, 'l2': 6, 'ratio': 0.65, 'mean': 28.15411281944587, 'std': 0.9102696481873485}
........................................
{'l1': 30, 'l2': 6, 'ratio': 0.9, 'mean': 28.324616327569156, 'std': 0.3718568601261248}
........................................
{'l1': 30, 'l2': 8, 'ratio': 0.4, 'mean': 24.53364136310025, 'std': 2.476468807832164}
........................................
{'l1': 30, 'l2': 8, 'ratio': 0.65, 'mean': 25.532412711

In [85]:
params_rs = pd.DataFrame.from_records(ret)
params_rs.sort_values(by='mean').tail(20)

,l1,l2,ratio,mean,std
28,35,12,0.65,27.464455,0.704372
37,40,8,0.65,27.493559,1.301865
2,30,4,0.90,27.576276,1.353543
44,40,12,0.90,27.687649,0.499640
29,35,12,0.90,27.705133,0.971466
23,35,8,0.90,27.718629,0.988859
43,40,12,0.65,27.770317,0.516750
19,35,6,0.65,27.799918,1.897795
41,40,10,0.90,27.934381,1.825360
32,40,4,0.90,28.022936,0.963807


In [82]:
params_rs = pd.DataFrame.from_records(ret)
params_rs.sort_values(by='mean').tail(20)

,l1,l2,iter,mean,std
16,30,16,2,25.427796,2.620284
11,30,6,3,25.787008,2.848802
2,20,6,3,25.801114,1.082514
13,30,11,2,26.635045,1.664329
15,30,16,1,26.669952,1.138085
22,40,11,2,26.730884,1.820168
25,40,16,2,26.897073,1.197451
7,20,16,2,27.105937,0.619543
4,20,11,2,27.229698,1.271680
3,20,11,1,27.522584,1.021802


In [77]:
params_rs = pd.DataFrame.from_records(ret)
params_rs.sort_values(by='mean').tail(20)

,l1,l2,iter,mean,std
3,9,3,4,4.995006,3.640686
0,9,2,4,6.191919,4.174401
5,9,3,6,7.362111,2.511720
1,9,2,5,7.450022,4.329654
6,9,4,4,7.679641,1.887059
8,9,4,6,7.754975,2.098707
2,9,2,6,7.777230,3.742215
4,9,3,5,7.969534,2.617259
7,9,4,5,8.363181,2.745448
12,9,6,4,9.152865,2.801934
